# Asignment 1bTask 2

Assignment 1 Question 2 Task 2 of stanford course, Tensorflow for Deep Learning Research. 
Assignment available here https://web.stanford.edu/class/cs20si/2017/assignments/a1.pdf

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xlrd
import math
import time

# Question 2 Task 2: Logistic regression on the <span style='color: red'>notMNIST</span> dataset
I guess the machine learning community is a bit sick of seeing MNIST pop up everywhere so
they created a similar dataset and literally named it notMNIST. Created by Yaroslav Bulatov, a
research engineer previously at Google and now at OpenAI, notMNIST is designed to look like
the classic MNIST dataset, but less ‘clean’ and extremely cute. The images are still 28x28 and
the there are also 10 labels, representing letters ‘A’ to ‘J’.


## Data Preprocessing 

#### Renaming and Skipping Corrupt Data

```python
# Rename all the images from 1 to last and skipping corrupt data
import os
import imageio
for folder in (['notMNIST_small','notMNIST_large']):
    for alphabet in (['A','B','C','D','E','F','G','H','I','J']): 
        print(alphabet,end='')  
        path = "E:/Datasets/notMNIST"+'/' +folder+'/'+alphabet 
        files = os.listdir(path)
        i = 0
        for file in files:
            try:
                img = imageio.imread(path+'/'+str(file)+'/')
                os.rename(os.path.join(path, file), os.path.join(path, str(i)+'.jpg'))  
                i = i+1
            except ValueError:    
                #os.remove(file)
                print("\nCorrupt Image was skipped.")
print('Done')
```

### Input Loading from Disk

In [112]:

import numpy as np
import imageio
import math
from sklearn.utils import shuffle

# Input: vector of class names, number of classes
# Convert an iterable of indices to one-hot encoded labels.
def one_hot(data, nb_classes):
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

#########################################################################
#############Load data batch by batch: if dataset is very large##########

# Renamge all the folder with number as each image name
# Function Name: create_batches
# Given data in local folder such that each class have different folder
# it return batch of data from a folder such that each class have end-start number of images
# INPUT: dataset_path: Path of local data folder
#        start: starting of image
#        end: last number of image
def create_batches(dataset_path,start,end):
    # to have equal number of samples from each alphabet/class
    alphabet_batch_size = int(batch_size/10)
    input_X = []
    input_Y = []
    # Go to each class folder, get equal number of samples,...
    # save them in list input_X/Y and then convert to np array 
    for alphabet in (['A','B','C','D','E','F','G','H','I','J']):
        path = dataset_path+'/' +alphabet
        alphabet_class = 0
        for img in range(start,end):
            input_X.append(imageio.imread(path+'/'+str(img)+'.jpg'))
            input_Y.append(alphabet_class)
        alphabet_class+=1    
    batch_X = np.asarray(input_X)
    batch_Y = np.asarray(input_Y)
    return batch_X,batch_Y

# Warning: batch_size can't be less than 10 (class size)
# Name: get_next_batch
# return batch of input and output taken from folders
# INPUT: dataset_path, batch_size and batch_number
def get_next_batch(dataset_path,batch_size,batch_number):
    num_classes = 10
    class_size = math.ceil(batch_size/num_classes)
    start = batch_number*class_size
    end = start + class_size
    X,Y = create_batches(dataset_path,start,end)
    batch_X = X[0:batch_size,:] 
    batch_Y = Y[0:batch_size]
    batch_X = np.reshape(batch_X,[batch_size,-1])
    batch_Y = one_hot(batch_Y,num_classes)
    return batch_X,batch_Y

##########################################################################
############# Load all the data from disk to RAM  ########################

# Load all the data into RAM, makes sures for 
def load_all_data(dataset_path,num):
    images_per_class = int(math.ceil(num/10)) +1
    num_classes = 10
    input_X = []
    input_Y = []
    # Go to each class folder, get equal number of samples,...
    # save them in list input_X/Y and then convert to np array 
    alphabet_class = 0
    for alphabet in (['A','B','C','D','E','F','G','H','I','J']):
        path = dataset_path+'/' +alphabet
        for img in range(1,images_per_class):
            input_X.append(imageio.imread(path+'/'+str(img)+'.jpg'))
            input_Y.append(alphabet_class)
        alphabet_class= alphabet_class+1    
    X1 = np.asarray(input_X)
    Y1 = np.asarray(input_Y)
    X = np.reshape(X1,[X1.shape[0],-1]).astype(float)
    Y = one_hot(Y1,num_classes)
    X,Y = shuffle(X,Y,random_state=0)
    return X,Y
# Similar to tf.mnist.get.next
def get_next_batch_RAM(X,Y,batch_size,batch_number):
    num_classes = 10
    # Indices to slice data into batches
    n = X.shape[0]
    start = batch_number*batch_size
    end = start + batch_size
    if(end-start==batch_size):
        batch_X = X[start:end,:]
        batch_Y = Y[start:end,:]
    else:
        batch_X = X[start:n,:]
        batch_Y = Y[start:n,:]
    return batch_X,batch_Y


In [113]:
# Test above functions
X1,Y1 = load_all_data('E:/Datasets/notMNIST/notMNIST_small',101)
x,y = get_next_batch_RAM(X1,Y1,79,1)


## Load data

In [114]:
# Load data into RAM
dataset_path_train = 'E:/Datasets/notMNIST/notMNIST_large'
dataset_path_test = 'E:/Datasets/notMNIST/notMNIST_small'
num_train = 40000
num_test = 10000

# Load all train and test data in RAM
print('Loading training data....')
X_train,Y_train = load_all_data(dataset_path_train,num_train)
print('Loading test data....')
X_test,Y_test = load_all_data(dataset_path_test,num_test)
print('Data sucessfully loaded into RAM')


Loading training data....
Loading test data....
Data sucessfully loaded into RAM


## Normalization

In [115]:
X_train/=255
X_test/=255

## Linear Regression with different regulrization

In [77]:
# Defing Model
# Define parameters 
tf.reset_default_graph()
learning_rate = 0.01
batch_size = 128
n_epochs = 1000
lamda = 0.001
LRnotMNIST = tf.Graph()
with LRnotMNIST.as_default():
    #Step 2: 
    X = tf.placeholder(tf.float32,[None,784],name='X_placeholder')
    Y = tf.placeholder(tf.int32,[None,10],name='Y')
    #Step 3:
    weights = tf.Variable(tf.random_normal(shape=[784,10],stddev=0.01),name='weights') 
    bias = tf.Variable(tf.zeros([1,10]),name='bias')

    #Step 4:
    logits = tf.matmul(X,weights) + bias
    
    #Step 5: 
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='loss')
    loss = tf.reduce_mean(entropy)#+ lamda*tf.norm(weights,ord=np.inf)
    # Step 6:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)  
    # Test 
    probs = tf.nn.softmax(logits)

In [78]:

# Parameters and variables
# Number of batches for test and train set
n_batches_train = int(math.ceil(num_train/batch_size))
n_batches_test = int(math.ceil(num_test/batch_size)) 
#print(n_batches_train)
# Run Session
with tf.Session(graph=LRnotMNIST) as sess:
    #writer = tf.summary.FileWriter('./graphs/assignment1_b',sess.graph)
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        total_loss = 0
        for j in range(n_batches_train):
            batch_number = j
            X_batch,Y_batch = get_next_batch_RAM(X_train,Y_train,batch_size,batch_number)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss +=loss_batch
            # Prints one hash after 10 batches
            if (j%10==0):
                print('#', end = "")
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches_train))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
    
# Test The model
    accuracy = 0
    Y_pred_1 =[]
    for i in range(n_batches_test):
        batch_number = i
        X_batch,Y_batch = get_next_batch_RAM(X_test,Y_test,batch_size,batch_number)
        probs_1 = sess.run(probs,feed_dict={X:X_batch})
        accuracy_batch = tf.equal(tf.argmax(probs_1,1),tf.argmax(Y_batch,1))
        correct_preds_batch= tf.reduce_sum(tf.cast(accuracy_batch,tf.float32))
        if (i%10==0):
                print('#', end = "")
        accuracy_1=sess.run(correct_preds_batch)   
        accuracy =accuracy_1+accuracy
    print('\nAccuracy:{0}'.format(accuracy/num_test))  
#writer.close()           

################################Average loss epoch 0:2403.7826407679354
################################Average loss epoch 1:1491.4853627750288
################################Average loss epoch 2:1420.677241023737
################################Average loss epoch 3:1354.668886324849
################################Average loss epoch 4:1307.0855305339583
################################Average loss epoch 5:1298.9271601960302
################################Average loss epoch 6:1317.6819969884123
################################Average loss epoch 7:1321.9140763450355
################################Average loss epoch 8:1322.2526849618735
################################Average loss epoch 9:1211.8555129176132
################################Average loss epoch 10:1257.7193030214157
################################Average loss epoch 11:1282.8830904732117
################################Average loss epoch 12:1233.3761758530102
################################Average loss epoch 13:1270.2294

## Results

|Test Set| Train Set| Regulrization|Normalization|Epochs|Accuracy|
|--------|----------|--------------|-------------|------|-------:|
|15000   |1000      |None          |Yes          |1     |  19.61 |
|=       |=         |None          |=            |1000  |  0.79  |
|=       |=         |L2            |=            |1000  |  79.38 |
|=       |=         |L1            |=            |1000  |   81   |
|=       |=         |inf           |=            |1000  |  79.41|
|=       |10000     |None          |=            |1000  |  78.49|
|=       |20000     |None          |=            |1000  |  77.8|
|=       |50000     |None          |=            |1000  |  67.09|
|=       |100000    |None          |=            |1000  |  52.37|
|18000   |500000    |None          |=            |1000  |  20.11|
|10000   |40000    |None          |Nor and Random Shuffuling            |1000  |  76.33|







<span style= "color:red">Weird Results</span>
Looks like noise have some effects on the effeciency. 


### Logistic Regression with SKlearn 

In [79]:
from sklearn import linear_model
from sklearn import metrics
LR = linear_model.LogisticRegression()
LR.fit(X_train,Y_train)
y_pred = LR.predict(X_test)
accuracy = metrics.accuracy_score(Y_test, y_pred, sample_weight=None)
print(accuracy)

ValueError: bad input shape (40000, 10)

|Train Set| Test Set| Accuracy Score| Regulrization|Normalization|
|---------|---------|---------------|--------------|------------:|
|1000     | 15000   |    75.15      | No           |Yes          |
|10000    | 15000   |    71.03      | No           |Yes          |
|100000   | =      |   89.35        | No           |Yes  | 

## Neural Network

Also experimented on DRAC1. Seems worse. A paper "[Generlization in Deep Learning](https://arxiv.org/abs/1710.05468)" introduces a new regulrization method they call DRAC1. For more details, read section 5 of the paper. 

In [138]:
# To avoid haveing different graphs colliding
tf.reset_default_graph()
learning_rate = 0.1
batch_size = 128
n_epochs = 1000
lamda = 0.01
weight_initializer_xavier = tf.contrib.layers.xavier_initializer()
# Neurons in each layer
X_shape_0 = 784 # X.shape[0]
n_outputs = 10
Neurons_1 = 500
Neurons_2 = 250
Neurons_3 = 100
Neurons_4 = 50

# graph 
NNnotMNIST = tf.Graph()
with NNnotMNIST.as_default():
    #plcae holders for input: 
    X = tf.placeholder(tf.float32,[None,X_shape_0],name='X')
    Y = tf.placeholder(tf.int32,[None,n_outputs],name='Y')
    # Layer 1
    w1 = tf.Variable(weight_initializer_xavier(shape=[X_shape_0,Neurons_1]),name='w1') 
    b1 = tf.Variable(tf.zeros([1,Neurons_1]),name='b1')
    z1 = tf.matmul(X,w1) + b1
    a1 = tf.nn.relu(z1)
    # Layer 2
    w2 = tf.Variable(weight_initializer_xavier(shape=[Neurons_1,Neurons_2]),name='w2') 
    b2 = tf.Variable(tf.zeros([1,Neurons_2]),name='b2')
    z2 = tf.matmul(a1,w2) + b2
    a2 = tf.nn.relu(z2)
    #Layer 3
    w3 = tf.Variable(weight_initializer_xavier(shape=[Neurons_2,Neurons_3]),name='w3') 
    b3 = tf.Variable(tf.zeros([1,Neurons_3]),name='b3')
    z3 = tf.matmul(a2,w3) + b3
    a3 = tf.nn.relu(z3)
    #Layer 4
    w4 = tf.Variable(weight_initializer_xavier(shape=[Neurons_3,Neurons_4]),name='w4') 
    b4 = tf.Variable(tf.zeros([1,Neurons_4]),name='b4')
    z4 = tf.matmul(a3,w4) + b4
    a4 = tf.nn.relu(z4)
    #Output Layer
    w_last = tf.Variable(weight_initializer_xavier(shape=[Neurons_4,n_outputs]),name='w_last') 
    b_last = tf.Variable(tf.zeros([1,n_outputs]),name='b_last')
    z_last = tf.matmul(a4,w_last) + b_last
    a_last = tf.nn.softmax(z_last)
    
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=a_last,name='loss')
    loss = tf.reduce_mean(entropy) 
    loss_DRAC1 = loss+ (lamda/batch_size)*tf.reduce_sum(tf.reduce_max(tf.abs(a3),axis=1))
    #loss_l1 = loss + 0.001* tf.norm(a3,ord=np.inf)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_DRAC1)
    # For test
    probs = tf.nn.softmax(a_last)
    #correct_preds = tf.equal(tf.argmax(probs,1),tf.argmax(Y_batch,1))
    #n_correct_preds = tf.reduce_sum(tf.cast(correct_preds,tf.float32))

In [139]:
# Run Session
tf.reset_default_graph()
with tf.Session(graph=NNnotMNIST) as sess:
    # to save the graph
    #writer = tf.summary.FileWriter('./graphs/assignment1_b',sess.graph)
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches_train = int(math.floor(num_train/batch_size))
    n_batches_test = int(math.floor(num_test/batch_size)) 
    
    #Training 
    for i in range(n_epochs):
        total_loss = 0
        # Passing through each batch
        for j in range(n_batches_train):
            batch_number = j
            X_batch,Y_batch = get_next_batch_RAM(X_train,Y_train,batch_size,batch_number)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss +=loss_batch
            # Prints one hash after 10 batches
            if (j%10==0):
                print('#', end = "")
                
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches_train))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
    
    # Testing
    accuracy = 0
    for i in range(n_batches_test):
        batch_number = i
        X_batch,Y_batch = get_next_batch_RAM(X_test,Y_test,batch_size,batch_number)
        probs_1 = sess.run(probs,feed_dict={X:X_batch})
        accuracy_batch = tf.equal(tf.argmax(probs_1,1),tf.argmax(Y_batch,1))
        correct_preds_batch= tf.reduce_sum(tf.cast(accuracy_batch,tf.float32))
        if (i%10==0):
                print('#', end = "")
        accuracy_1=sess.run(correct_preds_batch)   
        accuracy =accuracy_1+accuracy
    print('Accuracy{0}'.format(accuracy/num_test))    
#writer.close()    

################################Average loss epoch 0:1.8647535126178691
################################Average loss epoch 1:1.6504077770006962
################################Average loss epoch 2:1.629484420021375
################################Average loss epoch 3:1.6177937178275523
################################Average loss epoch 4:1.6090448873165326
################################Average loss epoch 5:1.6013436390039248
################################Average loss epoch 6:1.5943989138572643
################################Average loss epoch 7:1.5881024671670718
################################Average loss epoch 8:1.5828339744072695
################################Average loss epoch 9:1.5783448360669308
################################Average loss epoch 10:1.5741529388305469
################################Average loss epoch 11:1.5706635411733236
################################Average loss epoch 12:1.5673741690623455
################################Average loss epoch 13:1.564432

### Results 

- Layer 1 Neurons=100 Layer 2 Neurons = 50, epochs=100

|Train Set| Test Set| Accuracy Score| Regulrization|Normalization|
|---------|---------|---------------|--------------|------------:|
|1000     | 15000   |    0.26       | No           |Yes          |
|100000   | 15000   |    0.1        | No           |Yes          |

- Four Layer Network with 1000,500,250,50

|Test Set| Train Set| Regulrization|Normalization|Epochs|Accuracy|
|--------|----------|--------------|-------------|------|-------:|
|18000   |500000    |None          |=            |2  |  10.1|
|18000   |500000    |None          |=            |5  |  9.81|

- Four Layer Network with 500,250,100,50

|Test Set| Train Set| Regulrization|Normalization|Epochs|Accuracy|
|--------|----------|--------------|-------------|------|-------:|
|10000   |40000     |None          |=            |5  |  90|
|10000   |40000     |DRAC1 on a3          |=            |10,5  |  91.42,90.92|
|10000   |40000     |DRAC1 on a_last          |=            |5  |  73.57|
|10000   |40000     |l1 on w3          |=            |10  |  72.85|
|10000   |40000     |l1 on a3          |=            |5  |  10.06|
|10000   |40000     |linf on a3          |=            |5  |  77.64|